# Gráfico de Transformação - Auto Featuring - Experimento

Este é um componente que utiliza uma técnica para explorar as possíveis soluções orientadas por um gráfico para executar a AutoFeaturing. <br>

Este notebook apresenta:
- como usar o [SDK](https://platiagro.github.io/sdk/) para carregar datasets, salvar modelos e outros artefatos.
- como declarar parâmetros e usá-los para criar componentes reutilizáveis.

## Declaração de parâmetros e hiperparâmetros

Declare parâmetros com o botão  na barra de ferramentas.<br>
O parâmetro `dataset` identifica os conjuntos de dados. Você pode importar arquivos de dataset com o botão  na barra de ferramentas.

In [ ]:
# parâmetros
dataset = "/tmp/data/boston.csv" #@param {type:"string", label:"Base de dados", description:"Base de dados a ser utilizada."}
target = "medv" #@param {type:"feature", label:"Atributo alvo", description:"Seu modelo será treinado para prever os valores do alvo."}
date = None #@param {type:"feature", label:"Coluna de data", description:"Coluna com data que será utilizada para extrair novas características."}
group = ["age"] #@param {type:"feature", multiple:true, label:"Colunas para agrupar", description:"Colunas que serão utilizadas para agrupar e extrair novas características."}

budget = 20 #@param {type:"integer", label:"Limite de busca", description:"Parâmetro para aumentar ou diminuir a busca por soluções."}

## Acesso ao conjunto de dados

O conjunto de dados utilizado nesta etapa será o mesmo carregado através da plataforma.<br>
O tipo da variável retornada depende do arquivo de origem:
- [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) para CSV e compressed CSV: .csv .csv.zip .csv.gz .csv.bz2 .csv.xz
- [Binary IO stream](https://docs.python.org/3/library/io.html#binary-i-o) para outros tipos de arquivo: .jpg .wav .zip .h5 .parquet etc

In [ ]:
import pandas as pd

df = pd.read_csv(dataset)

## Acesso aos metadados do conjunto de dados

Utiliza a função `stat_dataset` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para carregar metadados. <br>
Por exemplo, arquivos CSV possuem `metadata['featuretypes']` para cada coluna no conjunto de dados (ex: categorical, numerical, or datetime).

In [ ]:
import numpy as np
from platiagro import stat_dataset

metadata = stat_dataset(name=dataset)
featuretypes = metadata["featuretypes"]
columns = df.columns
df_features = pd.DataFrame(featuretypes)

## Chamada do Auto Featuring

In [ ]:
!wget https://raw.githubusercontent.com/platiagro/projects/master/samples/transformation-graph/tgraph.py

In [ ]:
from tgraph import TGraph

tgraph = TGraph(df, df_features, target, group, date, budget)

# Pre-process
tgraph.preprocess()

# Autofeat
df, features, transformations = tgraph.auto_feat()

## Salva alterações no conjunto de dados

O conjunto de dados será salvo (e sobrescrito com as respectivas mudanças) localmente, no container da experimentação, utilizando a função `pandas.DataFrame.to_csv`.<br>

In [ ]:
# save dataset changes
df.to_csv(dataset, index=False)

## Salva modelo e outros resultados do treinamento

Escreve todos artefatos na pasta `/tmp/data/`. A plataforma guarda os artefatos desta pasta para usos futuros como implantação e comparação de resultados.

In [ ]:
from joblib import dump

artifacts = {
    "features": features,
    "transformations": transformations,
    "columns": list(columns),
}

dump(artifacts, "/tmp/data/transformation-graph.joblib")